In [4]:
import pandas as pd
import numpy as np
import csv
import json

In [9]:
df = pd.read_csv('clean-dataset.csv', encoding='utf-8')
df.columns

Index(['     1\tsong_id', 'title', 'artist', 'release_date', 'image_url',
       'timeSeries', 'summaries'],
      dtype='object')

### Nettoyage

In [13]:
df.rename(columns={'     1\tsong_id':'song_id'}, inplace=True)
df['song_id'] = df['song_id'].str.split('\t').str[1]
df.columns

Index(['song_id', 'title', 'artist', 'release_date', 'image_url', 'timeSeries',
       'summaries'],
      dtype='object')

### Analyse

In [16]:
json.loads(df['timeSeries'][0])

[{'date': '2024-04-29T00:00:00+00:00',
  'spotify-streams': 2398,
  'spotify-popularity': 8},
 {'date': '2024-04-30T00:00:00+00:00',
  'spotify-streams': 2404,
  'spotify-popularity': 8},
 {'date': '2024-05-01T00:00:00+00:00',
  'spotify-streams': 2414,
  'spotify-popularity': 8},
 {'date': '2024-05-02T00:00:00+00:00',
  'spotify-streams': 2417,
  'spotify-popularity': 8},
 {'date': '2024-05-03T00:00:00+00:00',
  'spotify-streams': 2428,
  'spotify-popularity': 8},
 {'date': '2024-05-04T00:00:00+00:00',
  'spotify-streams': 2434,
  'spotify-popularity': 8},
 {'date': '2024-05-05T00:00:00+00:00',
  'spotify-streams': 2446,
  'spotify-popularity': 8},
 {'date': '2024-05-06T00:00:00+00:00',
  'spotify-streams': 2456,
  'spotify-popularity': 8},
 {'date': '2024-05-07T00:00:00+00:00',
  'spotify-streams': 2458,
  'spotify-popularity': 8},
 {'date': '2024-05-08T00:00:00+00:00',
  'spotify-streams': 2464,
  'spotify-popularity': 8},
 {'date': '2024-05-09T00:00:00+00:00',
  'spotify-streams': 